# Логичтическая регрессия, метод опорных векторов, one-hot кодирование

### О задании

В этом задании вы изучите методы работы с категориальными переменными

In [27]:
%matplotlib inline
import pandas as pd

from sklearn.base import BaseEstimator
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

__Задание 1.__ Обучение логистической регрессии на реальных данных и оценка качества классификации.

**(2 балла)**


Загрузим данные с конкурса [Kaggle Porto Seguro’s Safe Driver Prediction](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction) (вам нужна только обучающая выборка). Задача состоит в определении водителей, которые в ближайший год воспользуются своей автомобильной страховкой (бинарная классификация). Но для нас важна будет не сама задача, а только её данные. При этом под нужды задания мы немного модифицируем датасет.

In [17]:
data = pd.read_csv('train.csv', index_col=0)
target = data.target.values
data = data.drop('target', axis=1)

Пересемплируем выборку так, чтобы положительных и отрицательных объектов в выборке было одинаковое число. Разделим на обучающую и тестовую выборки.

In [28]:
from sklearn.model_selection import train_test_split
import numpy as np
np.random.seed(910)
mask_plus = np.random.choice(np.where(target == 1)[0], 100000, replace=True)
mask_zero = np.random.choice(np.where(target == 0)[0], 100000, replace=True)

data = pd.concat((data.iloc[mask_plus], data.iloc[mask_zero]))
target = np.hstack((target[mask_plus], target[mask_zero]))

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.5)

Не забудьте отнормировать признаки (можно воспользоваться StandardScaler или сделать это вручную). Пока не будем обращать внимание на то, что некоторые признаки категориальные (этим мы займёмся позже).

In [32]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

start_data = data.copy()

def normalize(data): #dont forget to use this!!!
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)
    scaled_data = pd.DataFrame(scaled_data, columns=data.columns)
    return scaled_data

scaled_data = normalize(data)
scaled_data.describe()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,200000.000000,...,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05
mean,-4.661160e-17,1.312372e-16,1.601563e-16,8.242296e-17,-7.105427e-18,-4.462208e-17,4.348522e-17,6.785683e-17,-1.007550e-16,0.000000,...,1.319478e-16,-3.836931e-17,-5.023537e-17,-4.146017e-17,8.171241e-19,3.076650e-17,-3.581135e-17,-6.536993e-18,9.727330e-17,5.435652e-17
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003,...,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00
min,-9.905914e-01,-3.509020e+00,-1.636898e+00,-2.878048e+00,-1.000223e+00,-7.355743e-01,-6.474524e-01,-4.620329e-01,-4.647157e-01,-0.021685,...,-2.323999e+00,-1.201959e+00,-1.697466e+00,-2.748155e+00,-3.751168e-01,-1.307329e+00,-1.113178e+00,-6.352699e-01,-7.257531e-01,-4.246867e-01
25%,-9.905914e-01,-5.435048e-01,-9.067708e-01,-8.647089e-01,-3.344130e-01,-7.355743e-01,-6.474524e-01,-4.620329e-01,-4.647157e-01,-0.021685,...,-6.163894e-01,-3.694304e-01,-5.160407e-01,-5.625855e-01,-3.751168e-01,-1.307329e+00,-1.113178e+00,-6.352699e-01,-7.257531e-01,-4.246867e-01
50%,-4.948532e-01,-5.435048e-01,-1.766434e-01,-8.647089e-01,-3.344130e-01,-7.355743e-01,-6.474524e-01,-4.620329e-01,-4.647157e-01,-0.021685,...,-1.894870e-01,-3.694304e-01,7.467198e-02,-1.983240e-01,-3.751168e-01,7.649183e-01,8.983289e-01,-6.352699e-01,-7.257531e-01,-4.246867e-01
75%,4.966230e-01,9.392528e-01,9.185477e-01,1.148630e+00,-3.344130e-01,1.359482e+00,1.544515e+00,-4.620329e-01,-4.647157e-01,-0.021685,...,6.643178e-01,4.630982e-01,6.653846e-01,5.301991e-01,-3.751168e-01,7.649183e-01,8.983289e-01,1.574134e+00,1.377879e+00,-4.246867e-01
max,2.479576e+00,3.904768e+00,2.378802e+00,1.148630e+00,3.660446e+00,1.359482e+00,1.544515e+00,2.164348e+00,2.151853e+00,46.115719,...,5.360244e+00,5.458269e+00,5.981798e+00,5.265599e+00,2.665836e+00,7.649183e-01,8.983289e-01,1.574134e+00,1.377879e+00,2.354677e+00


,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,200000.000000,...,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05
mean,-4.661160e-17,1.312372e-16,1.601563e-16,8.242296e-17,-7.105427e-18,-4.462208e-17,4.348522e-17,6.785683e-17,-1.007550e-16,0.000000,...,1.319478e-16,-3.836931e-17,-5.023537e-17,-4.146017e-17,8.171241e-19,3.076650e-17,-3.581135e-17,-6.536993e-18,9.727330e-17,5.435652e-17
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003,...,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00
min,-9.905914e-01,-3.509020e+00,-1.636898e+00,-2.878048e+00,-1.000223e+00,-7.355743e-01,-6.474524e-01,-4.620329e-01,-4.647157e-01,-0.021685,...,-2.323999e+00,-1.201959e+00,-1.697466e+00,-2.748155e+00,-3.751168e-01,-1.307329e+00,-1.113178e+00,-6.352699e-01,-7.257531e-01,-4.246867e-01
25%,-9.905914e-01,-5.435048e-01,-9.067708e-01,-8.647089e-01,-3.344130e-01,-7.355743e-01,-6.474524e-01,-4.620329e-01,-4.647157e-01,-0.021685,...,-6.163894e-01,-3.694304e-01,-5.160407e-01,-5.625855e-01,-3.751168e-01,-1.307329e+00,-1.113178e+00,-6.352699e-01,-7.257531e-01,-4.246867e-01
50%,-4.948532e-01,-5.435048e-01,-1.766434e-01,-8.647089e-01,-3.344130e-01,-7.355743e-01,-6.474524e-01,-4.620329e-01,-4.647157e-01,-0.021685,...,-1.894870e-01,-3.694304e-01,7.467198e-02,-1.983240e-01,-3.751168e-01,7.649183e-01,8.983289e-01,-6.352699e-01,-7.257531e-01,-4.246867e-01
75%,4.966230e-01,9.392528e-01,9.185477e-01,1.148630e+00,-3.344130e-01,1.359482e+00,1.544515e+00,-4.620329e-01,-4.647157e-01,-0.021685,...,6.643178e-01,4.630982e-01,6.653846e-01,5.301991e-01,-3.751168e-01,7.649183e-01,8.983289e-01,1.574134e+00,1.377879e+00,-4.246867e-01
max,2.479576e+00,3.904768e+00,2.378802e+00,1.148630e+00,3.660446e+00,1.359482e+00,1.544515e+00,2.164348e+00,2.151853e+00,46.115719,...,5.360244e+00,5.458269e+00,5.981798e+00,5.265599e+00,2.665836e+00,7.649183e-01,8.983289e-01,1.574134e+00,1.377879e+00,2.354677e+00


Обучите логистическую регрессию с удобными для вас параметрами, примените регуляризацию, найдтие оптимум. Сделайте предсказание на тестовой части выборки. Замерьте качество.

In [33]:
def get_score(X_train, X_test, y_train, y_test):
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print(f"Accuracy:", accuracy)
    print(f"Precision:", precision)
    print(f"Recall:", recall)
    print(f"F1 Score:", f1)

In [34]:
%%time
get_score(*train_test_split(normalize(data), target, test_size=0.5, random_state=2243))

Accuracy: 0.58901
Precision: 0.5990521741029505
Recall: 0.5467410803268886
F1 Score: 0.5717024979418294
CPU times: user 976 ms, sys: 274 ms, total: 1.25 s
Wall time: 1.2 s


__Выводы__ в свободной форме: Модель логистической регрессии показывает неплохие результаты с точки зрения базового подхода к задаче. Регуляризация и правильная настройка параметров могут повысить эффективность модели. accuracy ~ 59%, precision ~ 60%, recall ~ 55% и F_score ~ 0.572 - это неплохо

__Задание 2.__ Изучение влияния регуляризатора на процесс обучения

__(2 балла)__

Проверьте на практике, как влияет регуляризатор на процесс обучения (убывание функции потерь на обучающей и отложенной выборках). Чтобы считать функцию потерь на отложенной выборке после каждой итерации, запускайте процесс обучения логистической регрессии с параметром $max\_iter=1$ и $w^{(0)}$, полученным на предыдущей итерации. Постройте два графика: на одном из них логистическая регрессия с коэффициентом регуляризации, равным 0, а на другом с некоторым разумным значением. На каждом графике одновременно должна быть и функция потерь для обучающей, и для тестовой выборки. Не забудьте сделать одинаковыми оси обоих графиков. Какие выводы вы можете сделать?

__Выводы:__

## Часть 2. Работа с категориальными переменными

В этой части мы научимся обрабатывать категориальные переменные, так как закодировать их в виде чисел недостаточно (это задаёт некоторый порядок, которого на категориальных переменных может и не быть). Существует два основных способа обработки категориальных значений:
- One-hot-кодирование
- Счётчики (CTR, mean-target кодирование, ...) — каждый категориальный признак заменяется на среднее значение целевой переменной по всем объектам, имеющим одинаковое значение в этом признаке.

Начнём с one-hot-кодирования. Допустим наш категориальный признак $f_j(x)$ принимает значения из множества $C=\{c_1, \dots, c_m\}$. Заменим его на $m$ бинарных признаков $b_1(x), \dots, b_m(x)$, каждый из которых является индикатором одного из возможных категориальных значений:
$$
b_i(x) = [f_j(x) = c_i]
$$

__Задание 1.__ Закодируйте все категориальные признаки с помощью one-hot-кодирования. Обучите логистическую регрессию и посмотрите, как изменилось качество модели (с тем, что было ранее). Измерьте время, потребовавшееся на обучение модели.

__(3 балла)__

In [35]:
categorical_features = [c for c in data.columns if c.endswith('_cat')]

encoder = OneHotEncoder(sparse_output=False, drop="first")
encoded_features = encoder.fit_transform(data[categorical_features])

data_one_hot = data.drop(categorical_features, axis=1)
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_features))

data_one_hot.reset_index(inplace=True, drop=True)
data_one_hot = pd.concat([data_one_hot, encoded_df], axis=1)

In [36]:
%%time
get_score(*train_test_split(normalize(data_one_hot), target, test_size=0.5, random_state=2243))

Accuracy: 0.5952
Precision: 0.6040279984540731
Recall: 0.5607335060793303
F1 Score: 0.5815761184156123
CPU times: user 4.85 s, sys: 1.11 s, total: 5.96 s
Wall time: 4.96 s


Как можно было заменить, one-hot-кодирование может сильно увеличивать количество признаков в датасете, что сказывается на памяти, особенно, если некоторый признак имеет большое количество значений. Эту проблему решает другой способ кодирование категориальных признаков — счётчики. Основная идея в том, что нам важны не сами категории, а значения целевой переменной, которые имеют объекты этой категории. Каждый категориальный признак мы заменим средним значением целевой переменной по всем объектам этой же категории:
$$
g_j(x, X) = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)][y_i = +1]}{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}
$$

__Задание 2.__ Закодируйте категориальные переменные с помощью счётчиков (ровно так, как описано выше без каких-либо хитростей). Обучите логистическую регрессию и посмотрите на качество модели на тестовом множестве. Сравните время обучения с предыдущим экспериментов. Заметили ли вы что-то интересное?

__(2 балла)__

In [37]:
CTR_data = data.copy()
import warnings, time
warnings.simplefilter("ignore")

categories = []

for v in data:
    if v[-3:] == "cat":
        categories.append(v)

for v in data:
    if v in categories:
        uniq = np.unique(data[v])
        for value in uniq:
            CTR_data[v][start_data[v] == value] = np.mean(target[start_data[v] == value])

CTR_data.describe()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,0.500000,4.483870,0.500000,0.500000,0.351100,0.295375,0.175920,0.177605,0.000470,...,5.443865,1.443745,2.873590,7.544455,0.123355,0.630875,0.553405,0.287530,0.345000,0.152800
std,2.017199,0.015959,2.739255,0.020419,0.050252,0.477315,0.456212,0.380753,0.382181,0.021674,...,2.342462,1.201163,1.692875,2.745287,0.328845,0.482569,0.497141,0.452612,0.475369,0.359796
min,0.000000,0.494915,0.000000,0.486218,0.480547,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.494915,2.000000,0.486218,0.480547,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.494915,4.000000,0.486218,0.480547,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,0.500783,7.000000,0.517318,0.480547,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,0.911215,11.000000,0.964072,0.697423,1.000000,1.000000,1.000000,1.000000,1.000000,...,18.000000,8.000000,13.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [38]:
%%time

get_score(*train_test_split(normalize(CTR_data), target, test_size=0.5, random_state=2243))

Accuracy: 0.59302
Precision: 0.601166317768189
Recall: 0.5609527606139127
F1 Score: 0.5803637713437269
CPU times: user 998 ms, sys: 258 ms, total: 1.26 s
Wall time: 1.28 s


__Вывод:__ При использовании счетчиков(CTR) и OneHotEncoding мы получаем похожие метрики, отличааются на ~0.1-0.3%, но при этом обучение при использовании счетчиков тратит гораздо меньше времени (1.3s VS 5s), чем OneHotEncoding


Отметим, что такие признаки сами по себе являются классификаторами и, обучаясь на них, мы допускаем "утечку" целевой переменной в признаки. Это ведёт к переобучению, поэтому считать такие признаки необходимо таким образом, чтобы при вычислении для конкретного объекта его целевая метка не использовалась. Это можно делать следующими способами:
- вычислять значение счётчика по всем объектам расположенным выше в датасете (например, если у нас выборка отсортирована по времени)
- вычислять по фолдам, то есть делить выборку на некоторое количество частей и подсчитывать значение признаков по всем фолдам кроме текущего (как делается в кросс-валидации)
- внесение некоторого шума в посчитанные признаки (необходимо соблюсти баланс между избавление от переобучения и полезностью признаков).

__Задание 3.__ Реализуйте корректное вычисление счётчиков двумя из трех вышеперчисленных способов, сравните. Снова обучите логистическую регрессию, оцените качество. Сделайте выводы.

__(3 балла)__

In [39]:
CTR_noise_data = data.copy()

for v in data:
    if v in categories:
        uniq = np.unique(data[v])
        for value in uniq:
            current = np.array(target[start_data[v] == value])
            CTR_noise_data[v][start_data[v] == value] = np.mean(current) + np.random.random(current.shape) * 0.02

CTR_noise_data.describe()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,0.509990,4.483870,0.510004,0.510006,0.351100,0.295375,0.175920,0.177605,0.000470,...,5.443865,1.443745,2.873590,7.544455,0.123355,0.630875,0.553405,0.287530,0.345000,0.152800
std,2.017199,0.016979,2.739255,0.021229,0.050581,0.477315,0.456212,0.380753,0.382181,0.021674,...,2.342462,1.201163,1.692875,2.745287,0.328845,0.482569,0.497141,0.452612,0.475369,0.359796
min,0.000000,0.494915,0.000000,0.486218,0.480547,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.501798,2.000000,0.494978,0.486378,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.508338,4.000000,0.503821,0.492231,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,0.514109,7.000000,0.525750,0.498058,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,0.931213,11.000000,0.983878,0.717422,1.000000,1.000000,1.000000,1.000000,1.000000,...,18.000000,8.000000,13.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [40]:
%%time

get_score(*train_test_split(normalize(CTR_noise_data), target, test_size=0.5, random_state=2243))

Accuracy: 0.59302
Precision: 0.6012485035060715
Recall: 0.5605740482359977
F1 Score: 0.5801992862005653
CPU times: user 1.23 s, sys: 307 ms, total: 1.53 s
Wall time: 1.15 s


In [41]:
fold_data = data.copy()

def get_sum (v, index, cumsum, cnt_blocks, n):
    part = n // cnt_blocks + 1
    l = (index // part) * part
    r = min(n, (index // part + 1) * (part))
    length = l + (n - r)
    sum = cumsum[l] + cumsum[n] - cumsum[r - 1]
    return sum / length

for v in categories:
    cnt_blocks = 8
    ans = []

    uniq = np.unique(data[v])
    cumsum = dict()
    cnt = dict()

    for u in uniq:
        np_data = np.array(target[data[v] == u])
        cumsum[u] = np.hstack((np.array([0]), np.cumsum(np_data)))

    for value in data[v]:
        if not value in cnt.items():
            cnt[value] = 0
        ans.append(get_sum(value, cnt[value], cumsum[value], cnt_blocks, len(cumsum[value]) - 1))
        cnt[value] += 1

    fold_data[v] = pd.Series(ans, index=data[v].index)

fold_data.describe()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,1.998215,0.428594,4.483870,0.428582,0.428601,0.351100,0.295375,0.175920,0.177605,0.000470,...,5.443865,1.443745,2.873590,7.544455,0.123355,0.630875,0.553405,0.287530,0.345000,0.152800
std,2.017199,0.018390,2.739255,0.023467,0.057497,0.477315,0.456212,0.380753,0.382181,0.021674,...,2.342462,1.201163,1.692875,2.745287,0.328845,0.482569,0.497141,0.452612,0.475369,0.359796
min,0.000000,0.422767,0.000000,0.412824,0.406342,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.422767,2.000000,0.412824,0.406342,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.422767,4.000000,0.412824,0.406342,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,3.000000,0.429577,7.000000,0.448371,0.406342,1.000000,1.000000,0.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,7.000000,0.903743,11.000000,0.965753,0.654545,1.000000,1.000000,1.000000,1.000000,1.000000,...,18.000000,8.000000,13.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [42]:
%%time

get_score(*train_test_split(normalize(fold_data), target, test_size=0.5, random_state=2243))

Accuracy: 0.59316
Precision: 0.601333162415075
Recall: 0.5610125573051624
F1 Score: 0.5804735192212506
CPU times: user 1 s, sys: 248 ms, total: 1.25 s
Wall time: 2.01 s


__Вывод:__ Разбиение по фолдам и добавление немного шума улучшают результаты